# Compute time to modify a single gene value on a cell type cluster, for each cell type for each dataset .

In [1]:
lapply(c("dplyr","Seurat","HGNChelper", "adverSCarial", "IRdisplay"), library, character.only = T)
options(warn=-1)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t




[[1]]
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
 [1] "Seurat"       "SeuratObject" "sp"           "dplyr"        "stats"       
 [6] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[11] "base"        

[[3]]
 [1] "HGNChelper"   "Seurat"       "SeuratObject" "sp"           "dplyr"       
 [6] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[11] "methods"      "base"        

[[4]]
 [1] "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject" "sp"          
 [6] "dplyr"        "stats"        "graphics"     "grDevices"    "utils"       
[11] "datasets"     "methods"      "base"        

[[5]]
 [1] "IRdisplay"    "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject"
 [6] "sp"           "dplyr"        "stats"        "graphics"     "grDevices"   
[11] "utils"        "datasets"     "methods"      "base"

In [2]:
bd <- "data/"

my_computer= "c1"

source("load_classifiers_v5.r")

In [3]:
c_paths <- list(pbmc_3k=list(so=paste0(bd, "v2//pbmc_test.rds"),
                             sce=paste0(bd, "v2//sce_pbmc_test.rds"),
                             df=paste0(bd, "v2//seurat_scaled_pbmc_test.txt")),
                kidney_10k=list(so=paste0(bd, "v5/data/sc/hgnc_kidney_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_kidney_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_kidney_10k_test.txt")),
                liver_10k=list(so=paste0(bd, "v5/data/sc/hgnc_liver_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_liver_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_liver_10k_test.txt")),
                axilla_10k=list(so=paste0(bd, "v5/data/sc/hgnc_axilla_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_axilla_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_axilla_10k_test.txt"))
               )

In [4]:
lsg <- list(pbmc_3k= read.table("data//v2/sign_genes.txt")$gene,
           liver_10k = read.table("data//v5/data/sign/sign_liver_10k.txt")$gene,
           kidney_10k = read.table("data//v5/data/sign/sign_kidney_10k.txt")$gene,
           axilla_10k = read.table("data//v5/data/sign/sign_axilla_10k.txt")$gene)

In [5]:
c_corres <- readRDS("data//v5/type_corres.rds")

In [6]:
ct_pbmc <- unique(unlist(lapply(grep("pbmc",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))
ct_pbmc

[1] "Memory CD4 T" "B"            "CD14+ Mono"   "NK"           "CD8 T"       
[6] "Naive CD4 T"  "FCGR3A+ Mono" "DC"           "Platelet"

In [7]:
ct_axilla <- unique(unlist(lapply(grep("axilla",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))
ct_axilla

[1] "malignant cell"                  "blood vessel endothelial cell"  
[3] "fibroblast"                      "macrophage"                     
[5] "blood vessel smooth muscle cell" "T cell"

In [8]:
ct_liver <- unique(unlist(lapply(grep("liver",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))
ct_liver

[1] "malignant cell"                  "endothelial cell"               
[3] "blood vessel smooth muscle cell" "macrophage"                     
[5] "monocyte"                        "mature NK T cell"

In [9]:
ct_kidney <- unique(unlist(lapply(grep("kidney",names(c_corres), value=T), function(x){
    unlist(strsplit(x,"__"))[3]
})))
ct_kidney

[1] "kidney epithelial cell"              
[2] "kidney cell"                         
[3] "podocyte"                            
[4] "epithelial cell of proximal tubule"  
[5] "mesenchymal cell"                    
[6] "kidney loop of Henle epithelial cell"

In [10]:
l_cts <- list("pbmc_3k"=ct_pbmc, "kidney_10k"=ct_kidney,
              "liver_10k"=ct_liver, "axilla_10k"=ct_axilla)

In [11]:
c_ids2run = c()
for (str_class in c("scType")){
    for (str_ds in names(l_cts)){
        for (str_ct in l_cts[[str_ds]]){
            c_ids2run <- c(c_ids2run, paste0(str_class,";",str_ds,";",str_ct))
        }
    }
}
c_ids2run <- c(paste0("c1;",c_ids2run), paste0("c2;", c_ids2run[length(c_ids2run):1]))
# c_ids2run <- c(paste0("c1;",c_ids2run), paste0("c2;", c_ids2run))
c_ids2run

[1] "c1;scType;pbmc_3k;Memory CD4 T"                           
 [2] "c1;scType;pbmc_3k;B"                                      
 [3] "c1;scType;pbmc_3k;CD14+ Mono"                             
 [4] "c1;scType;pbmc_3k;NK"                                     
 [5] "c1;scType;pbmc_3k;CD8 T"                                  
 [6] "c1;scType;pbmc_3k;Naive CD4 T"                            
 [7] "c1;scType;pbmc_3k;FCGR3A+ Mono"                           
 [8] "c1;scType;pbmc_3k;DC"                                     
 [9] "c1;scType;pbmc_3k;Platelet"                               
[10] "c1;scType;kidney_10k;kidney epithelial cell"              
[11] "c1;scType;kidney_10k;kidney cell"                         
[12] "c1;scType;kidney_10k;podocyte"                            
[13] "c1;scType;kidney_10k;epithelial cell of proximal tubule"  
[14] "c1;scType;kidney_10k;mesenchymal cell"                    
[15] "c1;scType;kidney_10k;kidney loop of Henle epithelial cell"
[16] "c1;scType;liver_10k;malignant cell"                       
[17] "c1;scType;liver_10k;endothelial cell"                     
[18] "c1;scType;liver_10k;blood vessel smooth muscle cell"      
[19] "c1;scType;liver_10k;macrophage"                           
[20] "c1;scType;liver_10k;monocyte"                             
[21] "c1;scType;liver_10k;mature NK T cell"                     
[22] "c1;scType;axilla_10k;malignant cell"                      
[23] "c1;scType;axilla_10k;blood vessel endothelial cell"       
[24] "c1;scType;axilla_10k;fibroblast"                          
[25] "c1;scType;axilla_10k;macrophage"                          
[26] "c1;scType;axilla_10k;blood vessel smooth muscle cell"     
[27] "c1;scType;axilla_10k;T cell"                              
[28] "c2;scType;axilla_10k;T cell"                              
[29] "c2;scType;axilla_10k;blood vessel smooth muscle cell"     
[30] "c2;scType;axilla_10k;macrophage"                          
[31] "c2;scType;axilla_10k;fibroblast"                          
[32] "c2;scType;axilla_10k;blood vessel endothelial cell"       
[33] "c2;scType;axilla_10k;malignant cell"                      
[34] "c2;scType;liver_10k;mature NK T cell"                     
[35] "c2;scType;liver_10k;monocyte"                             
[36] "c2;scType;liver_10k;macrophage"                           
[37] "c2;scType;liver_10k;blood vessel smooth muscle cell"      
[38] "c2;scType;liver_10k;endothelial cell"                     
[39] "c2;scType;liver_10k;malignant cell"                       
[40] "c2;scType;kidney_10k;kidney loop of Henle epithelial cell"
[41] "c2;scType;kidney_10k;mesenchymal cell"                    
[42] "c2;scType;kidney_10k;epithelial cell of proximal tubule"  
[43] "c2;scType;kidney_10k;podocyte"                            
[44] "c2;scType;kidney_10k;kidney cell"                         
[45] "c2;scType;kidney_10k;kidney epithelial cell"              
[46] "c2;scType;pbmc_3k;Platelet"                               
[47] "c2;scType;pbmc_3k;DC"                                     
[48] "c2;scType;pbmc_3k;FCGR3A+ Mono"                           
[49] "c2;scType;pbmc_3k;Naive CD4 T"                            
[50] "c2;scType;pbmc_3k;CD8 T"                                  
[51] "c2;scType;pbmc_3k;NK"                                     
[52] "c2;scType;pbmc_3k;CD14+ Mono"                             
[53] "c2;scType;pbmc_3k;B"                                      
[54] "c2;scType;pbmc_3k;Memory CD4 T"

In [12]:
df_times <- readRDS("data//v5/df_modif_times.rds")
df_times

,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1,-1,-1,-1,-1.0000000,-1
2,-1,-1,-1,-1,-1.0000000,-1
c1;scType;pbmc_3k;Memory CD4 T,-1,-1,-1,-1,-1.0000000,-1
c1;scType;pbmc_3k;B,-1,-1,-1,-1,-1.0000000,-1
c1;scType;pbmc_3k;CD14+ Mono,-1,-1,-1,-1,-1.0000000,-1
c1;scType;pbmc_3k;NK,-1,-1,-1,-1,-1.0000000,-1
c1;scType;pbmc_3k;CD8 T,-1,-1,-1,-1,-1.0000000,-1
c1;scType;pbmc_3k;Naive CD4 T,-1,-1,-1,-1,-1.0000000,-1
c1;scType;pbmc_3k;FCGR3A+ Mono,-1,-1,-1,-1,-1.0000000,-1


In [13]:
df_times[df_times$run_time != -1,]


,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
c2;scType;axilla_10k;T cell,-1,-1,-1,-1,6.9075842,-1
c2;scType;axilla_10k;blood vessel smooth muscle cell,-1,-1,-1,-1,7.4472220,-1
c2;scType;axilla_10k;macrophage,-1,-1,-1,-1,4.3051889,-1
c2;scType;axilla_10k;fibroblast,-1,-1,-1,-1,19.4733021,-1
c2;scType;axilla_10k;blood vessel endothelial cell,-1,-1,-1,-1,11.1207747,-1
c2;scType;axilla_10k;malignant cell,-1,-1,-1,-1,16.5087471,-1
c2;scType;liver_10k;mature NK T cell,-1,-1,-1,-1,8.0492675,-1
c2;scType;liver_10k;monocyte,-1,-1,-1,-1,2.1946607,-1
c2;scType;liver_10k;macrophage,-1,-1,-1,-1,8.5248697,-1


In [14]:
df_times['c2;MLP;axilla_10k;T cell',]

,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,NA,NA,NA,NA,NA,NA


In [15]:
myHourFormat <- function(start_time){
    diff_secs <- as.numeric(difftime(Sys.time(),start_time, units = "secs"))

    # Extraction des heures, minutes et secondes
    heures <- floor(diff_secs / 3600)
    minutes <- floor((diff_secs %% 3600) / 60)
    secondes <- round(diff_secs %% 60)

    # Formatage du temps écoulé
    temps_formate <- sprintf("%02d:%02d:%02d", heures, minutes, secondes)
    return(temps_formate)    
}


In [16]:
?advModifications

In [17]:
c_ids2run[1:10]

[1] "c1;scType;pbmc_3k;Memory CD4 T"             
 [2] "c1;scType;pbmc_3k;B"                        
 [3] "c1;scType;pbmc_3k;CD14+ Mono"               
 [4] "c1;scType;pbmc_3k;NK"                       
 [5] "c1;scType;pbmc_3k;CD8 T"                    
 [6] "c1;scType;pbmc_3k;Naive CD4 T"              
 [7] "c1;scType;pbmc_3k;FCGR3A+ Mono"             
 [8] "c1;scType;pbmc_3k;DC"                       
 [9] "c1;scType;pbmc_3k;Platelet"                 
[10] "c1;scType;kidney_10k;kidney epithelial cell"

In [18]:
only_myComp_ids <- grep(paste0("^",my_computer), c_ids2run, value=T)
for (str_id in only_myComp_ids){
    display(str_id)
    a_a = unlist(strsplit(str_id, ";"))
#     my_comp <- a_a[1]

    baseClass <- a_a[2]
    basen <- a_a[3]
    myCluster <- a_a[4]
    
    if (!str_id %in% rownames(df_times)){
        df_times[str_id,] = -1
    }
    if (nchar(as.character(df_times[str_id,"run_time"])) == 2){
        display(str_id)
# for (a_a in c_atts){
#     basen = a_a[1]
#     baseClass = a_a[2]
#     modif = a_a[3]
#     myCluster = a_a[4]
# for (basen in c_b2){
        message(basen)
        my_path <- paste0(bd, "v5/data/sc/hgnc_", basen, "_test.txt")
        my_path <- c_paths[[basen]]$df
        if (!file.exists(my_path)){
            message(my_path, " not available")
            next
        }
        df_pbmc_test <- read.table(my_path)
        expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
        clusters_df <- df_pbmc_test$y
        names(clusters_df) <- rownames(df_pbmc_test)

#         sign_path <- paste0(paste0(bd, "v5/data/sign/sign_", basen, ".txt"))
#         if (!file.exists(sign_path)){
#             message(sign_path, " not available, computing getSignGenes")
#             dfSG <- getSignGenes(expr_df, clusters_df, verbose=T)
#             write.table(dfSG$results, sign_path)
#         }
#         mySignGenes <- read.table(sign_path)

        message(baseClass)
        strClassifier <- paste0(baseClass, "_", basen)
        if ( baseClass == "scType" || basen=="pbmc_3k"){
            strClassifier <- baseClass
        }
        display(strClassifier)
        myClassifier <- classifList[[strClassifier]]$fct
        inputType <- classifList[[strClassifier]]$inputType

        display(inputType)
        mySlot <- classifList[[strClassifier]]$slot
        expr <- ""

        if(inputType == "data.frame"){
            expr <- expr_df
            clusters <- clusters_df
        }
        if(tolower(inputType) == "seurat"){
            my_path <- paste0(bd, "v5/data/sc/hgnc_", basen, "_test.rds")
            my_path <- c_paths[[basen]]$so
            if (!file.exists(my_path)){
                message(my_path, " not available")
                break
            }
            so_pbmc_test <- readRDS(my_path)

            clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_so) <- rownames(so_pbmc_test@meta.data)

            expr <- so_pbmc_test
            clusters <- clusters_so
        }
        if (tolower(inputType) == "singlecellexperiment"){
            my_path1 <- paste0(bd, "v5/data/sc/hgnc_",basen,"_test.rds")
            my_path1 <- c_paths[[basen]]$so
            my_path2 <- paste0(bd, "v5/data/sc/sce_hgnc_",basen,"_test.rds")
            my_path2 <- c_paths[[basen]]$sce
            if (!file.exists(my_path1) || !file.exists(my_path2)){
                message(my_path1, " or ", my_path2," not available")
                break
            }
            so_pbmc_test <- readRDS(my_path1)
            clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_so) <- rownames(so_pbmc_test@meta.data)

            sce_pbmc_test <- readRDS(my_path2)
            clusters_sce = so_pbmc_test@meta.data$chr_seurat_cluster
            names(clusters_sce) <- rownames(so_pbmc_test@meta.data)

            expr <- sce_pbmc_test
            clusters <- clusters_sce
        }
        message(myCluster)
        my_clusters = as.character(unname(clusters))
        if ( inputType == "SingleCellExperiment"){
            my_clusters = as.character(expr@colData$chr_seurat_cluster)
        }
        
        t1 <- Sys.time()
        class_res <- advModifications(expr, "GAPDH", my_clusters, myCluster)
        display(class_res)
        new_time <- difftime(Sys.time(), t1, units = "secs")
        df_times <- readRDS("data//v5/df_modif_times.rds")
        df_times[str_id, "run_time"] <- new_time
        saveRDS(df_times, "data//v5/df_modif_times.rds")
    } else {
        display(paste(str_id, " already done"))
    }
}


[1] "c1;scType;pbmc_3k;Memory CD4 T"

[1] "c1;scType;pbmc_3k;Memory CD4 T"

pbmc_3k

scType



[1] "scType"

[1] "data.frame"

...

axilla_10k

scType



[1] "scType"

[1] "data.frame"

T cell



<5312 x 25345> DelayedMatrix object of type "double":
                                                   ENSG00000238009 ...
HTAPP-878-SMP-7149-TST-channel1_GATGTTGCAAACGTGG-1      -0.2027578   .
HTAPP-878-SMP-7149-TST-channel1_CTTTCAAGTAGGTACG-1      -0.2027578   .
HTAPP-878-SMP-7149-TST-channel1_CACCGTTGTTCTGACA-1      -0.2027578   .
HTAPP-878-SMP-7149-TST-channel1_GAGTTTGCACAACGTT-1       1.5589332   .
HTAPP-878-SMP-7149-TST-channel1_GATGACTTCTTTGCTA-1      -0.2027578   .
                                               ...               .   .
HTAPP-878-SMP-7149-TST-channel2_GCGATCGGTACTCGCG-1      -0.2027578   .
HTAPP-878-SMP-7149-TST-channel2_TCACGGGCAGCTTTCC-1      -0.2027578   .
HTAPP-878-SMP-7149-TST-channel2_CTCCACAGTGTCATTG-1      -0.2027578   .
HTAPP-878-SMP-7149-TST-channel2_CATGAGTAGCTAAATG-1      -0.2027578   .
HTAPP-878-SMP-7149-TST-channel2_GGTTCTCGTGGCTCTG-1      -0.2027578   .
                                                   ENSG00000271254
HTAPP-878-SMP-7149-TST-chan

In [19]:
df_times

,modifs,classifier,ct,ds,run_time,misc
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-1,-1,-1,-1,-1.0000000,-1
2,-1,-1,-1,-1,-1.0000000,-1
c1;scType;pbmc_3k;Memory CD4 T,-1,-1,-1,-1,3.8227537,-1
c1;scType;pbmc_3k;B,-1,-1,-1,-1,0.5913293,-1
c1;scType;pbmc_3k;CD14+ Mono,-1,-1,-1,-1,0.4780073,-1
c1;scType;pbmc_3k;NK,-1,-1,-1,-1,0.4705262,-1
c1;scType;pbmc_3k;CD8 T,-1,-1,-1,-1,0.4895785,-1
c1;scType;pbmc_3k;Naive CD4 T,-1,-1,-1,-1,0.4634564,-1
c1;scType;pbmc_3k;FCGR3A+ Mono,-1,-1,-1,-1,0.4660211,-1
